Simple Gen AI App Using LangChain

In [11]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [12]:
## DATA INGESTION -- FROM the Website we need to scrape the data 

from langchain_community.document_loaders import WebBaseLoader 

In [13]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/self-hosted")
loader

In [14]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/self-hosted', 'title': 'Self-hosted LangSmith - Docs by LangChain', 'language': 'en'}, page_content='Self-hosted LangSmith - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationSelf-hostedSelf-hosted LangSmithGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewSet up LangSmithCloud (SaaS)HybridOverviewSetup guideSelf-hostedOverviewGet started by cloud providerSetup guidesEnable featuresConfigurationConnect external servicesPlatform auth & access controlSelf-hosted observabilityScripts for management tasksOn this pageSelf-host LangSmith observability and evaluationServicesStorage servicesSetup methodsSetup guidesEnable LangSmith DeploymentWorkflowStandalone ServerWorkflowSupported compute platformsSetup guideSelf-hostedSelf-hosted LangSmithCopy pageCopy pageImportant\nSelf-hosted LangSmith is

In [27]:
## Load Data --> Docs --> Divide our text into chunks -->vectors-->Vector Embeddings --> Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [28]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/self-hosted', 'title': 'Self-hosted LangSmith - Docs by LangChain', 'language': 'en'}, page_content='Self-hosted LangSmith - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationSelf-hostedSelf-hosted LangSmithGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewSet up LangSmithCloud (SaaS)HybridOverviewSetup guideSelf-hostedOverviewGet started by cloud providerSetup guidesEnable featuresConfigurationConnect external servicesPlatform auth & access controlSelf-hosted observabilityScripts for management tasksOn this pageSelf-host LangSmith observability and evaluationServicesStorage servicesSetup methodsSetup guidesEnable LangSmith DeploymentWorkflowStandalone ServerWorkflowSupported compute platformsSetup guideSelf-hostedSelf-hosted LangSmithCopy pageCopy pageImportant'),
 Document(metadata={'s

In [29]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [31]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [32]:
vectorstoredb

In [ ]:
## Query from a vector store db
query="LangSmith supports different self-hosted configurations depending on your scale"
result = vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
# Retrieval Chain , Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
        Answer the following question based on the provided context 
        <context>
        {context}
        </context>

        
    """
)